In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandas fiona shapely pyproj rtree
!pip install geopandas
!pip install contextily
import pandas as pd
import geopandas as gpd
import contextily as cx
import rtree
from shapely.geometry import Point, Polygon
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import random
import multiprocessing as mp
import dill
import progressbar
import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.7 MB 4.3 MB/s 
     |████████████████████████████████| 6.3 MB 45.5 MB/s 
     |████████████████████████████████| 1.0 MB 53.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 55 kB 5.0 MB/s 


## Load data

In [3]:
# Change dir accordingly
root_path = '/content/drive/MyDrive/BoxMigration/Geodata_Backup/'
fielddata_path = root_path + 'fields_withflare/'
flaredata_path = root_path + 'flaring_2020/'
result_path = root_path + 'flaring_inside_outside/'


In [6]:
#Troubleshooting: no special character in the folder name
# ! ls /content/drive/MyDrive/BoxMigration/Geodata_Backup/

In [4]:
# csv to geodf
def cvt_point_to_geodf(df,latname='Latitude',lonname='Longitude'):
    df['coords'] = list(zip(df[lonname],df[latname]))
    df['coords'] = df['coords'].apply(Point)
    gdf = gpd.GeoDataFrame(df, geometry='coords')
    return gdf

# load flare
def load_Flare(flaredata_path, country):
    flare = pd.read_csv(flaredata_path + "csv/" + country + ".csv")
    flare = cvt_point_to_geodf(flare, lonname='Longitude', latname='Latitude')
    flare = flare.rename(columns ={'Avg. temp., K':'Avg_temp_K'})
    flare = flare.rename(columns ={'Clear Obs.':'Clear_Obs'})
    flare = flare.rename(columns ={'BCM 2020':'BCM_2020'})
    flare = flare.set_crs(epsg=4326)
    
    return flare

# load field
def load_Field(fielddata_path, country):
    field = gpd.GeoDataFrame.from_file(fielddata_path + country + ".shp")
    field = field.set_crs(epsg=4326,allow_override=True)
    return field

# Join points inside to polygons
def join_Inside(flaredata_path, fielddata_path, country):
    flare = load_Flare(flaredata_path, country)
    field = load_Field(fielddata_path, country)
    flare['flare_count'] = 1
    field_in = gpd.tools.sjoin(field, flare, op='intersects', how='left')

    field_in = field_in.drop(columns=['index_right', 'Country_right', 'ISO Code', 'Catalog ID', 'ID 2020', 'Latitude', 'Longitude', 'Avg_temp_K', 'Ellipticity', 'Detection frequency 2020', 'Clear_Obs', 'Type'])

    field_in_groupby = field_in.groupby(['Number']).agg({'Country_left': 'first', 'Product_Ty': 'first', 'Number': 'first', 'N_Fldname': 'first', 'SUM_OIL_PR': 'first', 'SUM_GOR': 'first', 'BCM_2017': 'first', 'BCM_2018': 'first', 'BCM_2012': 'first', 'BCM_2013': 'first', 'BCM_2014': 'first', 'BCM_2015': 'first', 'BCM_2016': 'first', 'geometry': 'first', 'BCM_2020': np.sum, 'flare_count': np.sum})
    field_in_groupby = field_in_groupby.reset_index(level=0, drop=True).reset_index(drop=True)
    field_in_groupby = gpd.GeoDataFrame(field_in_groupby, crs="EPSG:4326", geometry=field_in_groupby["geometry"])
    field_in_groupby = field_in_groupby.rename(columns ={'Country_left':'Country'})
    
    return field_in_groupby

# Generate field buffer for outside flaring
def buffer_Field(fielddata_path, country, dist):
    field = load_Field(fielddata_path, country)
    # field_buffer = field.to_crs("EPSG:32634")
    field_buffer = field.to_crs("EPSG:3857")
    field_buffer.geometry = field_buffer.geometry.buffer(dist)
    field_buffer = field_buffer.to_crs("EPSG:4326")
    
    return field_buffer

# Remove flares within fields
def remove_Flarewithin(flaredata_path, fielddata_path, country):
    flare = load_Flare(flaredata_path, country)
    field = load_Field(fielddata_path, country)
    flare_out = gpd.tools.sjoin(field, flare, op='intersects', how='right')
    flare_out = flare_out[flare_out['index_left'].isna()]
    flare_out = flare_out.drop(columns=['index_left', 'Country_left', 'Product_Ty', 'Number', 'N_Fldname', 'SUM_OIL_PR', 'SUM_GOR', 'BCM_2017', 'BCM_2018', 'BCM_2012', 'BCM_2013', 'BCM_2014', 'BCM_2015', 'BCM_2016', 'BCM_2019'])
    flare_out = gpd.GeoDataFrame(flare_out, crs="EPSG:4326", geometry=flare_out["coords"])
    return flare_out

# Join points outside to polygons
def join_Outside(flaredata_path, fielddata_path, country, dist):
    field_buffer = buffer_Field(fielddata_path, country, dist)
    flare_out = remove_Flarewithin(flaredata_path, fielddata_path, country)
    flare_out['flare_count'] = 1
    field_out = gpd.tools.sjoin(field_buffer, flare_out, op='intersects', how='left')

    field_out = field_out.drop(columns=['index_right', 'Country_right', 'ISO Code', 'Catalog ID', 'ID 2020', 'Latitude', 'Longitude', 'Avg_temp_K', 'Ellipticity', 'Detection frequency 2020', 'Clear_Obs', 'Type'])

    field_out_groupby = field_out.groupby(['Number']).agg({'Country': 'first', 'Product_Ty': 'first', 'Number': 'first', 'N_Fldname': 'first', 'SUM_OIL_PR': 'first', 'SUM_GOR': 'first', 'BCM_2017': 'first', 'BCM_2018': 'first', 'BCM_2012': 'first', 'BCM_2013': 'first', 'BCM_2014': 'first', 'BCM_2015': 'first', 'BCM_2016': 'first', 'geometry': 'first', 'BCM_2020': np.sum, 'flare_count': np.sum})
    field_out_groupby = field_out_groupby.reset_index(level=0, drop=True).reset_index(drop=True)
    field_out_groupby = gpd.GeoDataFrame(field_out_groupby, crs="EPSG:4326", geometry=field_out_groupby["geometry"])
    return field_out_groupby

## Spatial join

In [22]:
# Test
country = "Norway"
dist = 5000

In [23]:
#Trouble shooting for file path: if the folder is shared, add a shortcut to boxmigration
flare =  load_Flare(flaredata_path, country)
flare.to_file(flaredata_path + country + ".shp", crs = "EPSG:4326")  
flare

,Country,ISO Code,Catalog ID,ID 2020,Latitude,Longitude,BCM_2020,Avg_temp_K,Ellipticity,Detection frequency 2020,Clear_Obs,Type,coords
0,Norway,NOR,NOR_UPS_2015_03.2210E_56.5458N_v0.2,5800,56.545778,3.220994,0.017037,1732.13,1.425670,74.846600,163,upstream oil,POINT (3.22099 56.54578)
1,Norway,NOR,NOR_UPS_2015_01.9066E_58.3678N_v0.2,5691,58.367750,1.906600,0.015246,1703.46,2.871970,72.388100,134,upstream oil,POINT (1.90660 58.36775)
2,Norway,NOR,NOR_UPS_2015_01.9031E_61.2948N_v0.2,5690,61.294820,1.903148,0.013150,1720.23,1.601500,67.768600,121,upstream oil,POINT (1.90315 61.29482)
3,Norway,NOR,NOR_UPS_2015_01.8546E_61.2541N_v0.2,5689,61.254100,1.854632,0.009880,1766.22,3.018900,50.862100,116,upstream oil,POINT (1.85463 61.25410)
4,Norway,NOR,NOR_UPS_2015_01.8319E_61.2052N_v0.2,5687,61.205190,1.831885,0.008295,1780.23,2.743800,38.016500,121,upstream oil,POINT (1.83188 61.20519)
5,Norway,NOR,NOR_UPS_2015_02.1908E_61.1759N_v0.2,5694,61.175895,2.190839,0.005674,1747.98,1.601500,20.805400,149,upstream oil,POINT (2.19084 61.17589)
6,Norway,NOR,NOR_UPS_2015_02.2014E_61.2023N_v0.2,5695,61.202290,2.201378,0.005051,1786.12,1.977590,35.507200,138,upstream oil,POINT (2.20138 61.20229)
7,Norway,NOR,NOR_UPS_2015_02.1424E_61.4479N_v0.2,5693,61.447910,2.142440,0.004806,1740.30,1.601500,13.725500,153,upstream oil,POINT (2.14244 61.44791)
8,Norway,NOR,NOR_UPS_2015_02.3874E_59.1905N_v0.2,5699,59.190547,2.387365,0.003520,1830.59,1.601500,9.146340,164,upstream oil,POINT (2.38736 59.19055)
9,Norway,NOR,NOR_UPS_2015_06.7260E_65.0633N_v0.2,5716,65.063285,6.725972,0.003185,1720.81,1.601500,3.597120,139,upstream oil,POINT (6.72597 65.06328)


In [24]:
field = load_Field(fielddata_path, country)
field

,Country,Number,Product_Ty,SUM_OIL_PR,SUM_GOR,BCM_2017,BCM_2018,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,BCM_2019,N_Fldname,geometry
0,Norway,0,GAS/CONDENSATE,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,SLEIPNER VEST,"MULTIPOLYGON (((1.66391 58.38956, 1.66391 58.3..."
1,Norway,1,GAS,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,GUNGNE,"MULTIPOLYGON (((1.88845 58.35192, 1.88845 58.3..."
2,Norway,2,GAS/CONDENSATE,15130.0,523.34,0.006436,0.012334,0.021169,0.019840,0.030303,0.011456,0.021853,0.015481,SLEIPNER OST,"MULTIPOLYGON (((1.98083 58.40733, 1.98083 58.4..."
3,Norway,3,OIL,1150.0,948.18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,HOD,"MULTIPOLYGON (((3.43037 56.17632, 3.43037 56.1..."
4,Norway,4,OIL,0.0,2667.17,0.000000,0.000000,0.000000,0.001124,0.000094,0.000169,0.000000,0.000000,GYDA,"POLYGON ((3.10259 56.83324, 3.10259 56.83324, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Norway,85,OIL/GAS,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,SINDRE,"POLYGON ((2.34711 61.23072, 2.34711 61.23072, ..."
86,Norway,86,OIL,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ODA,"POLYGON ((3.04298 57.05934, 3.04298 57.05934, ..."
87,Norway,87,None,0.0,530.20,0.004563,0.003413,0.001672,0.005168,0.005074,0.009362,0.005882,0.000000,Elli,"POLYGON ((2.35000 59.50488, 2.35351 59.50485, ..."
88,Norway,88,None,0.0,0.00,0.004563,0.003413,0.001672,0.005168,0.005074,0.009362,0.005882,0.000000,Elli Sor,"POLYGON ((2.32000 59.47488, 2.32351 59.47485, ..."


In [25]:
field_in_groupby = join_Inside(flaredata_path, fielddata_path, country)
field_in_groupby.to_file(result_path + country + "_in.shp", crs = "EPSG:4326")
field_in_groupby

,Country,Product_Ty,Number,N_Fldname,SUM_OIL_PR,SUM_GOR,BCM_2017,BCM_2018,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,geometry,BCM_2020,flare_count
0,Norway,GAS/CONDENSATE,0,SLEIPNER VEST,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((1.66391 58.38956, 1.66391 58.3...",0.000000,0.0
1,Norway,GAS,1,GUNGNE,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((1.88845 58.35192, 1.88845 58.3...",0.000000,0.0
2,Norway,GAS/CONDENSATE,2,SLEIPNER OST,15130.0,523.34,0.006436,0.012334,0.021169,0.019840,0.030303,0.011456,0.021853,"MULTIPOLYGON (((1.98083 58.40733, 1.98083 58.4...",0.015246,1.0
3,Norway,OIL,3,HOD,1150.0,948.18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((3.43037 56.17632, 3.43037 56.1...",0.000000,0.0
4,Norway,OIL,4,GYDA,0.0,2667.17,0.000000,0.000000,0.000000,0.001124,0.000094,0.000169,0.000000,"POLYGON ((3.10259 56.83324, 3.10259 56.83324, ...",0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Norway,OIL/GAS,85,SINDRE,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((2.34711 61.23072, 2.34711 61.23072, ...",0.000000,0.0
86,Norway,OIL,86,ODA,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((3.04298 57.05934, 3.04298 57.05934, ...",0.000000,0.0
87,Norway,None,87,Elli,0.0,530.20,0.004563,0.003413,0.001672,0.005168,0.005074,0.009362,0.005882,"POLYGON ((2.35000 59.50488, 2.35351 59.50485, ...",0.000181,1.0
88,Norway,None,88,Elli Sor,0.0,0.00,0.004563,0.003413,0.001672,0.005168,0.005074,0.009362,0.005882,"POLYGON ((2.32000 59.47488, 2.32351 59.47485, ...",0.000181,1.0


In [26]:
field_out_groupby = join_Outside(flaredata_path, fielddata_path, country, dist)
field_out_groupby.to_file(result_path + country + "_out.shp", crs = "EPSG:4326")
field_out_groupby

,Country,Product_Ty,Number,N_Fldname,SUM_OIL_PR,SUM_GOR,BCM_2017,BCM_2018,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,geometry,BCM_2020,flare_count
0,Norway,GAS/CONDENSATE,0,SLEIPNER VEST,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((1.59320 58.49408, 1.59327 58.49497, ...",0.0,0.0
1,Norway,GAS,1,GUNGNE,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((1.84013 58.30840, 1.83956 58.30889, ...",0.0,0.0
2,Norway,GAS/CONDENSATE,2,SLEIPNER OST,15130.0,523.34,0.006436,0.012334,0.021169,0.019840,0.030303,0.011456,0.021853,"POLYGON ((1.81649 58.36550, 1.81686 58.36678, ...",0.0,0.0
3,Norway,OIL,3,HOD,1150.0,948.18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((3.40137 56.21285, 3.40133 56.21313, ...",0.0,0.0
4,Norway,OIL,4,GYDA,0.0,2667.17,0.000000,0.000000,0.000000,0.001124,0.000094,0.000169,0.000000,"POLYGON ((2.94366 56.90866, 2.94324 56.90946, ...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Norway,OIL/GAS,85,SINDRE,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((2.28241 61.24165, 2.28198 61.24340, ...",0.0,0.0
86,Norway,OIL,86,ODA,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((2.99858 57.05587, 2.99802 57.05684, ...",0.0,0.0
87,Norway,None,87,Elli,0.0,530.20,0.004563,0.003413,0.001672,0.005168,0.005074,0.009362,0.005882,"POLYGON ((2.34910 59.52766, 2.35090 59.52766, ...",0.0,0.0
88,Norway,None,88,Elli Sor,0.0,0.00,0.004563,0.003413,0.001672,0.005168,0.005074,0.009362,0.005882,"POLYGON ((2.31910 59.49768, 2.32090 59.49768, ...",0.0,0.0


In [27]:
# Generate output and csv
field_in_groupby= field_in_groupby.rename(columns ={'BCM_2020':'BCM_2020_in'})
field_in_groupby['BCM_2020_out'] = field_out_groupby['BCM_2020']
field_in_groupby['BCM_2020'] = field_in_groupby['BCM_2020_in'] + field_in_groupby['BCM_2020_out']
field_in_groupby = field_in_groupby.drop(columns=['BCM_2020_in', 'flare_count', 'BCM_2020_out'])

field_in_groupby.to_file(result_path + "output/" + country + ".shp", crs = "EPSG:4326")
field_in_groupby.to_csv(result_path + 'csv_output/' + country + '.csv', index = False)
field_in_groupby

,Country,Product_Ty,Number,N_Fldname,SUM_OIL_PR,SUM_GOR,BCM_2017,BCM_2018,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,geometry,BCM_2020
0,Norway,GAS/CONDENSATE,0,SLEIPNER VEST,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((1.66391 58.38956, 1.66391 58.3...",0.000000
1,Norway,GAS,1,GUNGNE,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((1.88845 58.35192, 1.88845 58.3...",0.000000
2,Norway,GAS/CONDENSATE,2,SLEIPNER OST,15130.0,523.34,0.006436,0.012334,0.021169,0.019840,0.030303,0.011456,0.021853,"MULTIPOLYGON (((1.98083 58.40733, 1.98083 58.4...",0.015246
3,Norway,OIL,3,HOD,1150.0,948.18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((3.43037 56.17632, 3.43037 56.1...",0.000000
4,Norway,OIL,4,GYDA,0.0,2667.17,0.000000,0.000000,0.000000,0.001124,0.000094,0.000169,0.000000,"POLYGON ((3.10259 56.83324, 3.10259 56.83324, ...",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Norway,OIL/GAS,85,SINDRE,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((2.34711 61.23072, 2.34711 61.23072, ...",0.000000
86,Norway,OIL,86,ODA,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON ((3.04298 57.05934, 3.04298 57.05934, ...",0.000000
87,Norway,None,87,Elli,0.0,530.20,0.004563,0.003413,0.001672,0.005168,0.005074,0.009362,0.005882,"POLYGON ((2.35000 59.50488, 2.35351 59.50485, ...",0.000181
88,Norway,None,88,Elli Sor,0.0,0.00,0.004563,0.003413,0.001672,0.005168,0.005074,0.009362,0.005882,"POLYGON ((2.32000 59.47488, 2.32351 59.47485, ...",0.000181
